In [1]:
import os
import nibabel as nib
import numpy as np
import pandas as pd

In [2]:
cf_images_path = '/work/forkert_lab/erik/MACAW/cf_images'
original_image_path = '/work/forkert_lab/erik/T1_cropped'
output_path = '/work/forkert_lab/erik/JMI_paper_2'

# Composition and reversibility


In [3]:
original_image_eid = 4517026

In [4]:
#zero differences
axial_path_zero = '2_5Z_65_reverse'
twofive_path_zero = '2_5YZ_65_reverse'
pca3D_path_zero = 'PCA3D_15000_new_65_reverse'
autoencoder_path_zero = 'macaw_AE3D_6000_65_reverse'
vqvae_path_zero = 'macaw_vqvae8_50_nevecs_65_reverse'
# Reversebility
axial_path_10 =  '2_5Z_sex_0_reverse'
twofive_path_10 =  '2_5YZ_sex_0_reverse'
pca3D_path_10 =  'PCA3D_15000_new_sex_0_reverse'
autoencoder_path_10 = 'macaw_AE3D_6000_sex_0_reverse'
vqvae_path_10 = 'macaw_vqvae8_50_nevecs_sex_0_reverse'

comp_rever_paths_all = [axial_path_zero,
                       twofive_path_zero,
                       pca3D_path_zero,
                       autoencoder_path_zero,
                       vqvae_path_zero,
                       axial_path_10,
                       twofive_path_10,
                       pca3D_path_10,
                       autoencoder_path_10,
                       vqvae_path_10]

In [5]:
# getting original images
original_image = nib.load(os.path.join(original_image_path, str(original_image_eid) + ".nii.gz"))
original_image_np = original_image.get_fdata()
max_original = np.max(original_image_np)
right_shape = original_image_np.shape
right_shape

(150, 150, 100)

In [6]:
# getting cf images
ready_to_copy = []
to_cropy_images = []
ready_file_names = []
to_cropy_file_names = []
for each_path in comp_rever_paths_all:
    whole_path = os.path.join(cf_images_path,each_path)
    file_generated = [filename for filename in os.listdir(whole_path) if filename.startswith(f"{original_image_eid}")]
    image_load = nib.load(os.path.join(whole_path, file_generated[0])).get_fdata()
    if image_load.shape != right_shape:
        to_cropy_images.append(image_load)
        to_cropy_file_names.append(f"{each_path}_{original_image_eid}.nii.gz")
    else:
        ready_to_copy.append(image_load)
        ready_file_names.append(f"{each_path}_{original_image_eid}.nii.gz")
print(f"to_cropy_images: {len(to_cropy_images)}")
print(f"ready_to_copy: {len(ready_to_copy)}")

to_cropy_images: 4
ready_to_copy: 6


In [7]:
# cropping to same size of original image
for idx,each_generated_image in enumerate(to_cropy_images):
    generated_shape = each_generated_image.shape
    x_initial = int((generated_shape[0] - right_shape[0])/2)
    x_fim = x_initial +  right_shape[0]
    y_initial = int((generated_shape[1] - right_shape[1])/2)
    y_fim = y_initial +  right_shape[1]
    z_initial = int((generated_shape[2] - right_shape[2])/2)
    z_fim = z_initial +  right_shape[2]
    each_generated_image = each_generated_image[x_initial:x_fim,y_initial:y_fim,z_initial:z_fim] 
    print(each_generated_image.shape)
    ready_to_copy.append(each_generated_image)
    ready_file_names.append(to_cropy_file_names[idx])

(150, 150, 100)
(150, 150, 100)
(150, 150, 100)
(150, 150, 100)


In [8]:
# denormalize
for idx,one_image in enumerate(ready_to_copy):
    #minv = max_min_original[idx][z]['min']
    ready_to_copy[idx][:,:,:]= one_image[:,:,:] * max_original

In [9]:
for each_image in ready_to_copy:
    print(np.max(each_image))

1859.8780229251133
1853.3949414656963
1830.978811123874
1840.9674442468095
1840.9677734375
1840.9925724695204
1157.9921169545269
1656.3800191518385
1158.765385886596
1639.4615928024286


In [10]:
for idx, each_image in enumerate(ready_to_copy):
    ni_img = nib.Nifti1Image(each_image, original_image.affine, original_image.header)
    nib.save(ni_img,os.path.join(output_path,ready_file_names[idx]))

# Effectiviness

In [4]:
pred_path = '/home/erik.ohara/BrainAge'

In [5]:
df_orig = pd.read_csv(pred_path + '/predictions_real_test.csv')

In [6]:
df_orig.rename(columns={"Age": "Original_Age", 
                        "Prediction": "Original_Prediction", 
                        "ABSError": "Original_ABSError"},inplace=True)

In [7]:
df_cf_2_5 = pd.read_csv(pred_path + '/predictions_cf_age_2_5YZ_65.csv')

In [8]:
df_cf_analysis_2_5 = df_cf_2_5.set_index('EID').join(
                                            df_orig[['EID',
                                                     'Original_Age',
                                                     'Original_Prediction',
                                                     'Original_ABSError']].set_index('EID'), 
                                                on='EID', 
                                                how='left')

In [9]:
df_cf_analysis_2_5

Unnamed: 0  Age  Prediction  ABSError  ABSMEANError  Original_Age  \
EID                                                                          
1020415           0   65   69.359840  4.359840           0.0          70.0   
1025653           1   65   64.357056  0.642944           0.0          66.0   
1026490           2   65   58.531548  6.468452           0.0          56.0   
1027217           3   65   64.799194  0.200806           0.0          65.0   
1027582           4   65   57.977489  7.022511           0.0          55.0   
...             ...  ...         ...       ...           ...           ...   
6002830        2365   65   57.014378  7.985622           0.0          52.0   
6003437        2366   65   69.366486  4.366486           0.0          67.0   
6004251        2367   65   72.731667  7.731667           0.0          75.0   
6007875        2368   65   65.384827  0.384827           0.0          66.0   
6023202        2369   65   73.333626  8.333626           0.0          74.0   

         Original_Prediction  Original_ABSError  
EID                                              
1020415            70.551643           0.551643  
1025653            64.510208           1.489792  
1026490            57.647549           1.647549  
1027217            64.800774           0.199226  
1027582            55.508553           0.508553  
...                      ...                ...  
6002830            54.441154           2.441154  
6003437            69.560837           2.560837  
6004251            74.707329           0.292671  
6007875            65.767685           0.232315  
6023202            74.135048           0.135048  

[2370 rows x 8 columns]

In [21]:
achou_maior = False
achou_menor = False
error_maior = 99999
error_menor = 999999
for index, row in df_cf_analysis_2_5.iterrows():
    if (abs(row['Prediction'] - row['Original_Prediction']) > 3) and (row['ABSError']<2):
        if (row['Original_Age'] > row['Age']):
            if (row['ABSError'] < error_maior):
                achou_maior = True
                error_maior = row['ABSError']
                print(f"EID de mais velho: {index}")
                print(row)
                print()
        elif (row['Original_Age'] < row['Age']):
            if (row['ABSError'] < error_menor):
                achou_menor = True
                error_menor = row['ABSError']
                print(f"EID de mais novo: {index}")
                print(row)
                print()


EID de mais novo: 1069990
Unnamed: 0             28.000000
Age                    65.000000
Prediction             66.231369
ABSError                1.231369
ABSMEANError            0.000000
Original_Age           59.000000
Original_Prediction    63.063900
Original_ABSError       4.063900
Name: 1069990, dtype: float64

EID de mais novo: 1070244
Unnamed: 0             29.000000
Age                    65.000000
Prediction             63.914474
ABSError                1.085526
ABSMEANError            0.000000
Original_Age           58.000000
Original_Prediction    60.790791
Original_ABSError       2.790791
Name: 1070244, dtype: float64

EID de mais velho: 1128143
Unnamed: 0             49.000000
Age                    65.000000
Prediction             66.473579
ABSError                1.473579
ABSMEANError            0.000000
Original_Age           75.000000
Original_Prediction    69.674744
Original_ABSError       5.325256
Name: 1128143, dtype: float64

EID de mais novo: 1455398
Unnamed: 0

In [22]:
eid_younger = 2908349
eid_older = 3467967

In [31]:
# age 65
axial_path_65 = '2_5Z_65'
twofive_path_65 = '2_5YZ_65'
pca3D_path_65 = 'PCA3D_15000_new_65'
autoencoder_path_65 = 'macaw_AE3D_6000_65'
vqvae_path_65 = 'macaw_vqvae8_50nevecs_saving_65'

effec_age_paths_all = [axial_path_65,
                       twofive_path_65,
                       pca3D_path_65,
                       autoencoder_path_65,
                       vqvae_path_65]

## Younger

In [41]:
# getting original images
original_image_younger = nib.load(os.path.join(original_image_path, str(eid_younger) + ".nii.gz"))
original_image_younger_np = original_image_younger.get_fdata()
max_original_younger = np.max(original_image_younger_np)
right_shape_younger = original_image_younger_np.shape
right_shape_younger

(150, 150, 100)

In [32]:
# getting cf images
ready_to_copy = []
to_cropy_images = []
ready_file_names = []
to_cropy_file_names = []
for each_path in effec_age_paths_all:
    whole_path = os.path.join(cf_images_path,each_path)
    file_generated = [filename for filename in os.listdir(whole_path) if filename.startswith(f"{eid_younger}")]
    image_load = nib.load(os.path.join(whole_path, file_generated[0])).get_fdata()
    if image_load.shape != right_shape_younger:
        to_cropy_images.append(image_load)
        to_cropy_file_names.append(f"{each_path}_{eid_younger}.nii.gz")
    else:
        ready_to_copy.append(image_load)
        ready_file_names.append(f"{each_path}_{eid_younger}.nii.gz")
print(f"to_cropy_images: {len(to_cropy_images)}")
print(f"ready_to_copy: {len(ready_to_copy)}")

to_cropy_images: 2
ready_to_copy: 3


In [34]:
# cropping to same size of original image
for idx,each_generated_image in enumerate(to_cropy_images):
    generated_shape = each_generated_image.shape
    x_initial = int((generated_shape[0] - right_shape_younger[0])/2)
    x_fim = x_initial +  right_shape_younger[0]
    y_initial = int((generated_shape[1] - right_shape_younger[1])/2)
    y_fim = y_initial +  right_shape_younger[1]
    z_initial = int((generated_shape[2] - right_shape_younger[2])/2)
    z_fim = z_initial +  right_shape_younger[2]
    each_generated_image = each_generated_image[x_initial:x_fim,y_initial:y_fim,z_initial:z_fim] 
    print(each_generated_image.shape)
    ready_to_copy.append(each_generated_image)
    ready_file_names.append(to_cropy_file_names[idx])

(150, 150, 100)
(150, 150, 100)


In [42]:
# denormalize
for idx,one_image in enumerate(ready_to_copy):
    #minv = max_min_original[idx][z]['min']
    ready_to_copy[idx][:,:,:] = one_image[:,:,:] * max_original_younger

In [43]:
for each_image in ready_to_copy:
    print(np.max(each_image))

3586.8414125534473
3536.9825159875327
2742.7438581683964
1697.9411083068699
1809.1487040028878


In [45]:
for idx, each_image in enumerate(ready_to_copy):
    ni_img = nib.Nifti1Image(each_image, original_image_younger.affine, original_image_younger.header)
    nib.save(ni_img,os.path.join(output_path,ready_file_names[idx]))

In [46]:
diff_younger = []
for each_image in ready_to_copy:
    diff_younger.append(original_image_younger_np - each_image)

In [47]:
for idx, each_image in enumerate(diff_younger):
    ni_img = nib.Nifti1Image(each_image, original_image_younger.affine, original_image_younger.header)
    nib.save(ni_img,os.path.join(output_path,f"diff_{ready_file_names[idx]}"))

## older

In [49]:
# getting original images
original_image_older = nib.load(os.path.join(original_image_path, str(eid_older) + ".nii.gz"))
original_image_older_np = original_image_older.get_fdata()
max_original_older = np.max(original_image_older_np)
right_shape_older = original_image_older_np.shape
right_shape_older

(150, 150, 100)

In [50]:
# getting cf images
ready_to_copy = []
to_cropy_images = []
ready_file_names = []
to_cropy_file_names = []
for each_path in effec_age_paths_all:
    whole_path = os.path.join(cf_images_path,each_path)
    file_generated = [filename for filename in os.listdir(whole_path) if filename.startswith(f"{eid_older}")]
    image_load = nib.load(os.path.join(whole_path, file_generated[0])).get_fdata()
    if image_load.shape != right_shape_older:
        to_cropy_images.append(image_load)
        to_cropy_file_names.append(f"{each_path}_{eid_older}.nii.gz")
    else:
        ready_to_copy.append(image_load)
        ready_file_names.append(f"{each_path}_{eid_older}.nii.gz")
print(f"to_cropy_images: {len(to_cropy_images)}")
print(f"ready_to_copy: {len(ready_to_copy)}")

to_cropy_images: 2
ready_to_copy: 3


In [51]:
# cropping to same size of original image
for idx,each_generated_image in enumerate(to_cropy_images):
    generated_shape = each_generated_image.shape
    x_initial = int((generated_shape[0] - right_shape_older[0])/2)
    x_fim = x_initial +  right_shape_older[0]
    y_initial = int((generated_shape[1] - right_shape_older[1])/2)
    y_fim = y_initial +  right_shape_older[1]
    z_initial = int((generated_shape[2] - right_shape_older[2])/2)
    z_fim = z_initial +  right_shape_older[2]
    each_generated_image = each_generated_image[x_initial:x_fim,y_initial:y_fim,z_initial:z_fim] 
    print(each_generated_image.shape)
    ready_to_copy.append(each_generated_image)
    ready_file_names.append(to_cropy_file_names[idx])

(150, 150, 100)
(150, 150, 100)


In [52]:
# denormalize
for idx,one_image in enumerate(ready_to_copy):
    #minv = max_min_original[idx][z]['min']
    ready_to_copy[idx][:,:,:] = one_image[:,:,:] * max_original_older

In [53]:
for each_image in ready_to_copy:
    print(np.max(each_image))

2558.8056161672575
2584.5609478432743
2009.579875253694
1368.802102773072
1448.1391632009181


In [54]:
for idx, each_image in enumerate(ready_to_copy):
    ni_img = nib.Nifti1Image(each_image, original_image_older.affine, original_image_older.header)
    nib.save(ni_img,os.path.join(output_path,ready_file_names[idx]))

In [55]:
diff_older = []
for each_image in ready_to_copy:
    diff_older.append(original_image_older_np - each_image)

In [56]:
for idx, each_image in enumerate(diff_older):
    ni_img = nib.Nifti1Image(each_image, original_image_older.affine, original_image_older.header)
    nib.save(ni_img,os.path.join(output_path,f"diff_{ready_file_names[idx]}"))

## Sex

In [57]:
male_eid = 3788604

In [61]:
# all females
axial_path_sex = '2_5Z_sex_0'
twofive_path_sex = '2_5YZ_sex_0'
pca3D_path_sex = 'PCA3D_15000_new_sex_females'
autoencoder_sex = 'macaw_AE3D_6000_sex_0'
vqvae_path_sex = 'macaw_vqvae8_50nevecs_saving_sex_0'

effec_sex_paths_all = [axial_path_sex,
                       twofive_path_sex,
                       pca3D_path_sex,
                       autoencoder_sex,
                       vqvae_path_sex]

In [58]:
# getting original images
original_image_male = nib.load(os.path.join(original_image_path, str(male_eid) + ".nii.gz"))
original_image_male_np = original_image_male.get_fdata()
max_original_male = np.max(original_image_male_np)
right_shape_male = original_image_male_np.shape
right_shape_male

(150, 150, 100)

In [62]:
# getting cf images
ready_to_copy = []
to_cropy_images = []
ready_file_names = []
to_cropy_file_names = []
for each_path in effec_sex_paths_all:
    whole_path = os.path.join(cf_images_path,each_path)
    file_generated = [filename for filename in os.listdir(whole_path) if filename.startswith(f"{male_eid}")]
    image_load = nib.load(os.path.join(whole_path, file_generated[0])).get_fdata()
    if image_load.shape != right_shape_male:
        to_cropy_images.append(image_load)
        to_cropy_file_names.append(f"{each_path}_{male_eid}.nii.gz")
    else:
        ready_to_copy.append(image_load)
        ready_file_names.append(f"{each_path}_{male_eid}.nii.gz")
print(f"to_cropy_images: {len(to_cropy_images)}")
print(f"ready_to_copy: {len(ready_to_copy)}")

to_cropy_images: 2
ready_to_copy: 3


In [63]:
# cropping to same size of original image
for idx,each_generated_image in enumerate(to_cropy_images):
    generated_shape = each_generated_image.shape
    x_initial = int((generated_shape[0] - right_shape_male[0])/2)
    x_fim = x_initial +  right_shape_male[0]
    y_initial = int((generated_shape[1] - right_shape_male[1])/2)
    y_fim = y_initial +  right_shape_male[1]
    z_initial = int((generated_shape[2] - right_shape_male[2])/2)
    z_fim = z_initial +  right_shape_male[2]
    each_generated_image = each_generated_image[x_initial:x_fim,y_initial:y_fim,z_initial:z_fim] 
    print(each_generated_image.shape)
    ready_to_copy.append(each_generated_image)
    ready_file_names.append(to_cropy_file_names[idx])

(150, 150, 100)
(150, 150, 100)


In [64]:
# denormalize
for idx,one_image in enumerate(ready_to_copy):
    #minv = max_min_original[idx][z]['min']
    ready_to_copy[idx][:,:,:] = one_image[:,:,:] * max_original_male

In [65]:
for each_image in ready_to_copy:
    print(np.max(each_image))

2079.210764336167
2106.45360993105
1872.7803535707062
1338.7623331113718
1310.067815844377


In [66]:
for idx, each_image in enumerate(ready_to_copy):
    ni_img = nib.Nifti1Image(each_image, original_image_male.affine, original_image_male.header)
    nib.save(ni_img,os.path.join(output_path,ready_file_names[idx]))

In [67]:
diff_male = []
for each_image in ready_to_copy:
    diff_male.append(original_image_male_np - each_image)

In [68]:
for idx, each_image in enumerate(diff_male):
    ni_img = nib.Nifti1Image(each_image, original_image_male.affine, original_image_male.header)
    nib.save(ni_img,os.path.join(output_path,f"diff_{ready_file_names[idx]}"))